In [ ]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_pickle("strava_data.pickle")

In [ ]:
df['start_date_local'] = pd.to_datetime(df['start_date_local']).dt.date

In [ ]:
runs = df.loc[df['type'] == 'Run']

In [ ]:
def speed_to_pace(x):
    # First, convert m/s to km/min
    x = x * (60/1000)
    # Second, convert km/min to min/km.
    x = 1 / x
    return x

In [ ]:
# Create pace from speed.
runs.loc[:, 'average_pace'] = speed_to_pace(runs['average_speed'])
runs.loc[:, 'max_pace'] = speed_to_pace(runs['max_speed'])
runs.loc[:, 'hr_ratio'] = runs['average_heartrate'] / runs['average_pace']
# Partition runs into different pace clusters.
runs.loc[:, 'pace_cat'] = pd.qcut(runs['average_pace'], 4, labels=False)

In [ ]:
runs

In [ ]:
runs.plot(x='start_date_local', y='average_heartrate', style='.')

In [ ]:
runs.plot(x='start_date_local', y='max_heartrate', style='.')

In [ ]:
runs.plot(x='start_date_local', y='hr_ratio', style='.')

In [ ]:
fig, ax = plt.subplots()
for key, grp in runs.groupby(['pace_cat']):
    ax = grp.plot(ax=ax, kind='line', x='start_date_local', y='average_heartrate', label=key, style='.')

## Compile plots into single plot

In [ ]:
plt.rcParams.update({'font.size': 14})

fig, axs = plt.subplots(3, 1, figsize=(10,15))
fig.tight_layout(pad=3)

runs.plot(x='start_date_local', y='average_heartrate', style='.', ax=axs[0])
axs[0].set_title("Average heart rate per run")

runs.plot(x='start_date_local', y='max_heartrate', style='.', ax=axs[1])
axs[1].set_title("Max heart rate per run")

runs.plot(x='start_date_local', y='hr_ratio', style='.', ax=axs[2])
axs[2].set_title("Average heart rate/pace per run")

for ax in axs:
    ax.set_xlabel("run date")
    ax.set_ylabel("Heart rate [bpm]")
    
axs[2].set_ylabel("Heart rate/pace [bpm * km / min]")
fig.savefig("2021_heart_rates.png")